In [1]:
import os
import requests
import logging
import pandas as pd
from bs4 import BeautifulSoup
import re

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Base URL for linking
base_url = "https://lawphil.net/statutes/bataspam/"
start_url = "https://lawphil.net/statutes/bataspam/bataspam.html"

# Directory to save the data files
data_dir = "Data/Statutes/Batas Pambansa"
os.makedirs(data_dir, exist_ok=True)

def fetch_page(url):
    """Fetches the HTML content of a given URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        logger.error(f"Failed to fetch page: {url}. Error: {e}")
        return None

def parse_main_page(html):
    """Parses the main page to find the desired table with the list of acts."""
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'id': 's-menu'})
    links = []
    
    if not table:
        logger.error("No table with id 's-menu' found on the page.")
        return links
    
    # Process rows within the table
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        if len(columns) < 2:
            continue

        number_link = columns[0].find('a')
        title = columns[1].text.strip()
        
        if number_link and number_link['href']:
            full_link = base_url + number_link['href']
            links.append((full_link, title))
    
    logger.info(f"Found {len(links)} links in the table.")
    return links

def parse_act_page(url):
    """Parses the act page to retrieve the Number, Title, and Data."""
    html = fetch_page(url)
    if not html:
        return None, None, None

    soup = BeautifulSoup(html, 'html.parser')
    blockquote = soup.find('blockquote')
    
    if not blockquote:
        logger.warning(f"No blockquote found on page: {url}")
        return None, None, None

    # Log the entire blockquote content for debugging
    logger.debug(f"Blockquote content for {url}: {blockquote.text.strip()}")

    paragraphs = blockquote.find_all('p')
    
    if len(paragraphs) < 2:
        logger.warning(f"Insufficient paragraphs in blockquote at: {url}")
        return None, None, None

    # Extract Number from the first <p> or <b> element
    number_element = paragraphs[0].find(['p', 'b'])
    number = number_element.text.strip() if number_element else ''
    number = re.sub(r'[\[\]]', '', number)  # Remove square brackets

    # Extract Title from the second <p> or <b> element
    title_element = paragraphs[1].find(['p', 'b'])
    title = title_element.text.strip() if title_element else ''
    
    # Extract all text inside the blockquote for Data
    data = blockquote.text.strip()
    
    return number, title, data



def save_to_csv(number, title, data):
    """Saves the act information to a CSV file in the data directory."""
    file_path = os.path.join(data_dir, f"{number}.csv")
    df = pd.DataFrame([[number, title, data]], columns=["Number", "Title", "Data"])
    df.to_csv(file_path, index=False, encoding='utf-8')
    logger.info(f"Data saved to {file_path}")

def main():
    main_page_html = fetch_page(start_url)
    if not main_page_html:
        logger.error("Failed to retrieve main page.")
        return

    act_links = parse_main_page(main_page_html)
    for url, title in act_links:
        logger.info(f"Processing URL: {url}")
        number, title, data = parse_act_page(url)
        if number and title and data:
            save_to_csv(number, title, data)
        else:
            logger.warning(f"Skipping due to incomplete data for URL: {url}")

if __name__ == "__main__":
    main()


2024-11-14 16:47:52,353 - INFO - Found 890 links in the table.
2024-11-14 16:47:52,354 - INFO - Processing URL: https://lawphil.net/statutes/bataspam/bp1985/bp_889_1985.html
2024-11-14 16:47:52,608 - WARNING - Skipping due to incomplete data for URL: https://lawphil.net/statutes/bataspam/bp1985/bp_889_1985.html
2024-11-14 16:47:52,609 - INFO - Processing URL: https://lawphil.net/statutes/bataspam/bp1985/bp_888_1985.html
2024-11-14 16:47:52,866 - WARNING - Skipping due to incomplete data for URL: https://lawphil.net/statutes/bataspam/bp1985/bp_888_1985.html
2024-11-14 16:47:52,866 - INFO - Processing URL: https://lawphil.net/statutes/bataspam/bp1985/bp_887_1985.html
2024-11-14 16:47:53,130 - WARNING - Skipping due to incomplete data for URL: https://lawphil.net/statutes/bataspam/bp1985/bp_887_1985.html
2024-11-14 16:47:53,131 - INFO - Processing URL: https://lawphil.net/statutes/bataspam/bp1985/bp_886_1985.html
2024-11-14 16:47:53,377 - WARNING - Skipping due to incomplete data for URL:

KeyboardInterrupt: 